# Apply direct stiffness method to solve for the larger (general) case of a truss structure

In [22]:
import numpy as np
import matplotlib.pyplot as plt
import math

# constants
E =  1 #200*10**9 #(N/m^2) 
A =  1 #0.005 #(m^2) 
L = 10 #1 #(m)

# degree of freedom
nodes = 6
nDof = nodes * 2


# restraint nodes
restraint_nDof = [6,7,10,11]

# quadrants alias
Quadrants = {
    'Q11': 0,
    'Q12': 1,
    'Q21': 2,
    'Q22': 3
}

## Utilise

In [ ]:
def bar_globalstiffness(E, A, L, theta, node1, node2, tag):
    """
    Calculate the global stiffness matrix for a bar element in a 2D truss structure.
    Parameters:
    E (float): Young's modulus of the material.
    A (float): Cross-sectional area of the bar.
    L (float): Length of the bar.
    theta (float): Angle of the bar with respect to the horizontal axis in degrees.
    Returns:
    numpy.ndarray: A 4x4 global stiffness matrix for the bar element.
    """

    theta = math.radians(theta)  # Convert theta from degrees to radians
    c = math.cos(theta)
    s = math.sin(theta)
    k = E * A / L
    K = np.array([[c**2, c*s, -c**2, -c*s],
                  [c*s, s**2, -c*s, -s**2],
                  [-c**2, -c*s, c**2, c*s],
                  [-c*s, -s**2, c*s, s**2]]) * k
    
    element = {
        'tag' : tag,
        'node' : [node1, node2],
        'global_stiffness' : K
    }
    return element

elm_1 = bar_globalstiffness(E, A, L, 60, 1, 2, A)
print(elm_1["node"])

def structure_primaryStiffness (nDof, Elms):
    """
    Calculate the primary stiffness matrix for a 2D truss structure.
    Parameters:
    nDof (int): Number of degrees of freedom in the structure.
    Elms (numpy.ndarray): An array of elements in the structure.
    Returns:
    numpy.ndarray: A nDof x nDof primary stiffness matrix for the structure.
    """
    prime = np.zeros((nDof, nDof))
    for i in range(0, nDof, 2): # index of rows
        for j in range(0, nDof, 2): # index of columns 
            dummy_quadrant = np.zeros((2,2))
            for elm in Elms:
                if i/2 == j/2 and i/2 == elm['node'][0]:
                    dummy_quadrant += elm['quadrants'][Quadrants['Q11']]
                
                elif i/2 == j/2 and i/2 == elm['node'][1]:
                    dummy_quadrant += elm['quadrants'][Quadrants['Q22']]

                elif i/2 == elm['node'][0] and j/2 == elm['node'][1]:
                    dummy_quadrant += elm['quadrants'][Quadrants['Q12']]
                
                elif i/2 == elm['node'][1] and j/2 == elm['node'][0]:
                    dummy_quadrant += elm['quadrants'][Quadrants['Q21']]

                prime[i:i+2, j:j+2] = dummy_quadrant
    return prime

[1, 2]


## Declaring the global stiffness matrix for each bar element

In [ ]:
elm_A = bar_globalstiffness(E, A, L, 60, 0, 1, 'A')  # NODE 0-1 
elm_B = bar_globalstiffness(E, A, L, 0, 1, 2, 'B')   # NODE 1-2  
elm_C = bar_globalstiffness(E, A, L, 300, 2, 3, 'C') # NODE 2-3
elm_D = bar_globalstiffness(E, A, L, 180, 3, 4, 'D') # NODE 3-4
elm_E = bar_globalstiffness(E, A, L, 0, 0, 4, 'E')   # NODE 0-4
elm_F = bar_globalstiffness(E, A, L, 300, 1, 4, 'F') # NODE 1-4
elm_G = bar_globalstiffness(E, A, L, 240, 2, 4, 'G') # NODE 2-4
elm_H = bar_globalstiffness(E, A, 5, 270, 4, 5, 'H') # NODE 4-5

ELM_ARR = np.array([elm_A,
            elm_B,
            elm_C,
            elm_D,
            elm_E,
            elm_F,
            elm_G,
            elm_H])

for elm in ELM_ARR:
    Q11 = elm['global_stiffness'][:2, :2]
    Q12 = elm['global_stiffness'][:2, 2:]
    Q21 = elm['global_stiffness'][2:, :2]
    Q22 = elm['global_stiffness'][2:, 2:]
    elm['quadrants'] = [Q11, Q12, Q21, Q22]


In [27]:
prime_matrix = structure_primaryStiffness(nDof, ELM_ARR)
print(np.round(prime_matrix,2))


[[ 0.13  0.04 -0.03 -0.04  0.    0.    0.    0.   -0.1   0.    0.    0.  ]
 [ 0.04  0.08 -0.04 -0.08  0.    0.    0.    0.    0.    0.    0.    0.  ]
 [-0.03 -0.04  0.15  0.   -0.1   0.    0.    0.   -0.03  0.04  0.    0.  ]
 [-0.04 -0.08  0.    0.15  0.    0.    0.    0.    0.04 -0.08  0.    0.  ]
 [ 0.    0.   -0.1   0.    0.15  0.   -0.03  0.04 -0.03 -0.04  0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.15  0.04 -0.08 -0.04 -0.07  0.    0.  ]
 [ 0.    0.    0.    0.   -0.03  0.04  0.13 -0.04 -0.1   0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.04 -0.08 -0.04  0.08  0.   -0.    0.    0.  ]
 [-0.1   0.   -0.03  0.04 -0.03 -0.04 -0.1   0.    0.25  0.   -0.   -0.  ]
 [ 0.    0.    0.04 -0.08 -0.04 -0.07  0.   -0.    0.    0.35 -0.   -0.2 ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.   -0.   -0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.   -0.   -0.2   0.    0.2 ]]
